In [100]:
import pandas as pd
import numpy as np
import itertools as it 
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from ast import literal_eval

# Guide and Ideas for Capstone Project

We begin this guide by downloading the data:

In [101]:
file = 8
resultsFile = "data/results03.csv"

In [102]:
inputColumnName = "f" + str(file)
outputColumnName = "f" + str(file) + "_output"
df = pd.read_csv(resultsFile)
input = df[inputColumnName]
output = df[outputColumnName]
print("Input=", input)
print("Output=", output)
df.head()


Input= 0    [0.067737 0.079147 0.027514 0.046544 0.461641 ...
1    [0.081284 0.094976 0.033017 0.055853 0.369313 ...
2    [0.081284 0.094976 0.033017 0.055853 0.553969 ...
Name: f8, dtype: object
Output= 0    9.799602
1    9.696555
2    9.817107
Name: f8_output, dtype: float64


,Unnamed: 0,timestamp,student_id,f1,f2,f3,f4,f5,f6,f7,f8,f1_output,f2_output,f3_output,f4_output,f5_output,f6_output,f7_output,f8_output
0,24,2024-05-05 12:53:51,534,[0.877228 0.8796 ],[0.562109 0.999999],[0.463899 0.489275 0.408212],[0.462212 0.386621 0.366643 0.298809],[0.269027 0.999999 0.999999 0.999999],[0.582549 0.123754 0.586041 0.832796 0.045121],[0.046316 0.393338 0.197938 0.174495 0.336343 ...,[0.067737 0.079147 0.027514 0.046544 0.461641 ...,2.380231e-87,0.038342,-0.024742,-1.917274,4462.544669,-0.595724,1.286160,9.799602
1,45,2024-05-07 07:25:14,534,[0.877228 0.758174],[0.843164 0.999999],[0.509702 0.39142 0.489854],[0.426175 0.38531 0.370372 0.358571],[0.322832 0.999999 0.999999 0.999999],[0.466039 0.148505 0.468833 0.666237 0.036097],[0.046316 0.393338 0.197938 0.174495 0.360367 ...,[0.081284 0.094976 0.033017 0.055853 0.369313 ...,-1.758338e-55,-0.068606,-0.010482,0.563498,4484.415611,-0.678115,1.708912,9.696555
2,82,2024-05-13 06:14:44,534,[0.785666 0.5864 ],[0.769352 0.892841],[0.371119 0.39142 0.489854],[0.34094 0.363106 0.444446 0.430285],[0.387398 0.999999 0.999999 0.999999],[0.466039 0.148505 0.703249 0.999355 0.036097],[0.037053 0.472006 0.15835 0.139596 0.319182 ...,[0.081284 0.094976 0.033017 0.055853 0.553969 ...,-9.346589e-20,0.171941,-0.008681,0.180900,4528.747422,-0.551035,1.033901,9.817107


In [103]:
def load_data(dim):
    X = np.load('data/initial_data/function_' + str(dim) +'/initial_inputs.npy')
    y = np.load('data/initial_data/function_' + str(dim) +'/initial_outputs.npy')
    return X,y

X, y = load_data(file)
dimension = np.shape(X)[1] 
print("Original X(shape)=", X.shape, "y(shape)=", y.shape)

for item in input:
     item_with_comma = "[" + item.replace(" 0.",", 0.")  + "]"  
     row = np.array(literal_eval(item_with_comma))
     X = np.append(X, np.array(row), axis=0)

for item in output:
    y = np.append(y, float(item))

print("Updated X(shape)=", X.shape, "y(shape)=", y.shape)


Original X(shape)= (40, 8) y(shape)= (40,)
Updated X(shape)= (43, 8) y(shape)= (43,)


In [104]:
X

array([[0.60499445, 0.29221502, 0.90845275, 0.35550624, 0.20166872,
        0.57533801, 0.31031095, 0.73428138],
       [0.17800696, 0.56622265, 0.99486184, 0.21032501, 0.32015266,
        0.70790879, 0.63538449, 0.10713163],
       [0.00907698, 0.81162615, 0.52052036, 0.07568668, 0.26511183,
        0.09165169, 0.59241515, 0.36732026],
       [0.50602816, 0.65373012, 0.36341078, 0.17798105, 0.0937283 ,
        0.19742533, 0.7558269 , 0.29247234],
       [0.35990926, 0.24907568, 0.49599717, 0.70921498, 0.11498719,
        0.28920692, 0.55729515, 0.59388173],
       [0.77881834, 0.0034195 , 0.33798313, 0.51952778, 0.82090699,
        0.53724669, 0.5513471 , 0.66003209],
       [0.90864932, 0.0622497 , 0.23825955, 0.76660355, 0.13233596,
        0.99024381, 0.68806782, 0.74249594],
       [0.58637144, 0.88073573, 0.74502075, 0.54603485, 0.00964888,
        0.74899176, 0.23090707, 0.09791562],
       [0.76113733, 0.85467239, 0.38212433, 0.33735198, 0.68970832,
        0.30985305, 0.631379

In [105]:
y


array([7.3987211 , 7.00522736, 8.45948162, 8.28400781, 8.60611679,
       8.54174792, 7.32743458, 7.29987205, 7.95787474, 5.59219339,
       7.85454099, 6.79198578, 8.97655402, 7.3790829 , 9.598482  ,
       8.15998319, 7.13162397, 6.76796253, 7.43374407, 9.01307515,
       7.31089382, 5.84106731, 9.14163949, 8.81755844, 6.45194313,
       8.83074505, 9.34427428, 6.88784639, 8.04221254, 7.69236805,
       7.92375877, 8.42175924, 8.2780624 , 7.11345716, 6.40258841,
       8.47293632, 7.97768459, 7.46087219, 7.43659353, 9.18300525,
       9.79960224, 9.69655457, 9.81710696])

In [106]:
if (dimension == 2):
    x1 =X[:,0]
    x2 =X[:,1] 
    plt.scatter(x1, x2, c=y)
    for (i, j, text) in zip(x1, x2, y):
        plt.annotate(format(text, ".2e"),  # This is the text to use for the annotation
                 (i, j),  # This is the point to label
                 textcoords="offset points",  # how to position the text
                 xytext=(0,10),  # distance from text to points (x,y)
                 ha='right')  # horizontal alignment can be left, right or center
    plt.show()

# Upper Confidence Bound

A second alternative would be to use Bayesian Optimization and consider an Upper Confidence Bound acquisition function: 

In [107]:
# Add here hyper-parameters
gpr = GaussianProcessRegressor()
gpr.fit(X, y)



GaussianProcessRegressor()

In [108]:
def find_maxiumum(X, y):
    maximum_of_X = []
    maximum_so_far = -1e99
    for index in range(len(y)):
        if (y[index] > maximum_so_far):
            maximum_so_far = y[index]
            maximum_of_X = []
            for i in range(dimension):
                maximum_of_X.append(X[index, i])
    return np.array(maximum_of_X), maximum_so_far
    
#X_sample =np.array([[1,10], [2,20], [3,30], [4, 40]])
#Xmax, ymax = find_maxiumum(X_sample, [3,2,5,1])
Xmax, ymax = find_maxiumum(X, y)
XmaxLower = Xmax - (Xmax * 0.2)
XmaxUpper = Xmax + (Xmax * 0.2)
print("maximum X=", Xmax, ", maximum y=", ymax)

sizes = [ [2, 100],
          [3,  80],
          [4,  60],
          [5,  30],
          [6,  15],
          [7,  10],
          [8,  8]]

#Generate each dimension 
size = sizes[dimension - 2][1]

data = []
for item in range(dimension):
    data.append(np.linspace(max(XmaxLower[item], 0 + 1e-10), min(XmaxUpper[item], 1 - 1e-10), size))

x_grid_points = np.array(data)

print("data-points=", x_grid_points.shape)
for dim in range(dimension):
    print("data-points[", dim + 1, "]=", x_grid_points[dim][:5], "...", x_grid_points[dim][-1] )



maximum X= [0.081284 0.094976 0.033017 0.055853 0.553969 0.769013 0.334839 0.775708] , maximum y= 9.8171069550181
data-points= (8, 8)
data-points[ 1 ]= [0.0650272 0.069672  0.0743168 0.0789616 0.0836064] ... 0.0975408
data-points[ 2 ]= [0.0759808 0.081408  0.0868352 0.0922624 0.0976896] ... 0.11397120000000001
data-points[ 3 ]= [0.0264136  0.02830029 0.03018697 0.03207366 0.03396034] ... 0.0396204
data-points[ 4 ]= [0.0446824 0.047874  0.0510656 0.0542572 0.0574488] ... 0.0670236
data-points[ 5 ]= [0.4431752  0.47483057 0.50648594 0.53814131 0.56979669] ... 0.6647628000000001
data-points[ 6 ]= [0.6152104 0.659154  0.7030976 0.7470412 0.7909848] ... 0.9228156
data-points[ 7 ]= [0.2678712  0.28700486 0.30613851 0.32527217 0.34440583] ... 0.4018068
data-points[ 8 ]= [0.6205664  0.66489257 0.70921874 0.75354491 0.79787109] ... 0.9308495999999999


In [109]:
X_grid = np.fromiter(it.chain(*it.product(*x_grid_points)), dtype=float).reshape(-1,dimension)
print("X_grid=", X_grid.shape, " = ", X_grid[0:5,:], "...", X_grid[-1,:])


X_grid= (16777216, 8)  =  [[0.0650272  0.0759808  0.0264136  0.0446824  0.4431752  0.6152104
  0.2678712  0.6205664 ]
 [0.0650272  0.0759808  0.0264136  0.0446824  0.4431752  0.6152104
  0.2678712  0.66489257]
 [0.0650272  0.0759808  0.0264136  0.0446824  0.4431752  0.6152104
  0.2678712  0.70921874]
 [0.0650272  0.0759808  0.0264136  0.0446824  0.4431752  0.6152104
  0.2678712  0.75354491]
 [0.0650272  0.0759808  0.0264136  0.0446824  0.4431752  0.6152104
  0.2678712  0.79787109]] ... [0.0975408 0.1139712 0.0396204 0.0670236 0.6647628 0.9228156 0.4018068
 0.9308496]


In [110]:
#X_grid = np.array(X_grid)
mean, std = gpr.predict(X_grid, return_std = True)
ucb = mean + 1.96 * std

In [111]:
print("UCB=", ucb.shape, ", value=", ucb[:5])
print("mean=", mean.shape, ", value=", mean[:5])
#print("mean + std(1.96)=", mean.shape, ", value=", (mean + std * 1.96)[:5])


UCB= (16777216,) , value= [9.78431144 9.80985757 9.83091556 9.84690268 9.85706785]
mean= (16777216,) , value= [9.66910767 9.69969363 9.72048085 9.73117844 9.73150799]


In [112]:
idx_max = np.argmax(ucb)
next_query = X_grid[idx_max]
print(next_query)
best_numbers = []
for item in next_query:
    best_numbers.append(min(max(item, 0.000001), 0.999999))
print(best_numbers)


[0.0975408  0.1139712  0.0396204  0.0670236  0.4431752  0.659154
 0.30613851 0.84219726]
[0.0975408, 0.11397120000000001, 0.0396204, 0.0670236, 0.44317520000000005, 0.6591539999999999, 0.30613851428571426, 0.8421972571428571]


Doing this every week, we expect to be able to find the first of the two modes! This should would be enough to provide a good solution to the problem. To really verify if we found the maximum, we would need to find a second mode, however, we may be unlucky and simply never find it. This is an important part of Machine Learning: in theory we might want optimal, perfect solutions, but in practice most of the advancements can be done with simple solutions and careful consideration of the data available to us.

In [113]:
np.array2string(np.array(best_numbers), precision=6, separator='-', floatmode='fixed', formatter={'float': '{:0.6f}'.format})

'[0.097541-0.113971-0.039620-0.067024-0.443175-0.659154-0.306139-0.842197]'